In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud,STOPWORDS
import io
import base64
from matplotlib import rc,animation
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.basemap import Basemap
import folium
import folium.plugins
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
print(os.listdir("../input"))


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Data

The data is provided by [Home Credit](http://www.homecredit.net/about-us.aspx), a service dedicated to provided lines of credit (loans) to the unbanked population. Predicting whether or not a client will repay a loan or have difficulty is a critical business need, and Home Credit is hosting this competition on Kaggle to see what sort of models the machine learning community can develop to help them in this task. 

There are 7 different sources of data:

* application_train/application_test: the main training and testing data with information about each loan application at Home Credit. Every loan has its own row and is identified by the feature `SK_ID_CURR`. The training application data comes with the `TARGET` indicating 0: the loan was repaid or 1: the loan was not repaid. 
* bureau: data concerning client's previous credits from other financial institutions. Each previous credit has its own row in bureau, but one loan in the application data can have multiple previous credits.
* bureau_balance: monthly data about the previous credits in bureau. Each row is one month of a previous credit, and a single previous credit can have multiple rows, one for each month of the credit length. 
* previous_application: previous applications for loans at Home Credit of clients who have loans in the application data. Each current loan in the application data can have multiple previous loans. Each previous application has one row and is identified by the feature `SK_ID_PREV`. 
* POS_CASH_BALANCE: monthly data about previous point of sale or cash loans clients have had with Home Credit. Each row is one month of a previous point of sale or cash loan, and a single previous loan can have many rows.
* credit_card_balance: monthly data about previous credit cards clients have had with Home Credit. Each row is one month of a credit card balance, and a single credit card can have many rows.
* installments_payment: payment history for previous loans at Home Credit. There is one row for every made payment and one row for every missed payment. 

This diagram shows how all of the data is related:

![image](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

Moreover, we are provided with the definitions of all the columns (in `HomeCredit_columns_description.csv`) and an example of the expected submission file. 

In this notebook, we will stick to using only the main application training and testing data. Although if we want to have any hope of seriously competing, we need to use all the data, for now we will stick to one file which should be more manageable. This will let us establish a baseline that we can then improve upon. With these projects, it's best to build up an understanding of the problem a little at a time rather than diving all the way in and getting completely lost! 

# missing data fonksiyon

In [ ]:
# get the number of missing data points per column
def missing_data(df):
    total = df.isnull().sum()
    percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
    types = df.dtypes
    unieq = df.select_dtypes(include = [object,float,int]).apply(pd.Series.nunique, axis = 0)
    df_mis = pd.concat([total, percent ,unieq,types], axis=1, keys=['Sum_Mis', 'Per_Mis','Unieq', 'Types'])
    
    return df_mis.head(len(df_mis))

In [ ]:
prev = pd.read_csv('/kaggle/input/home-credit-default-risk/previous_application.csv')
pos = pd.read_csv('/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv')
ins = pd.read_csv('/kaggle/input/home-credit-default-risk/installments_payments.csv')
cc = pd.read_csv('/kaggle/input/home-credit-default-risk/credit_card_balance.csv')

In [ ]:
def heatmap(df):
    df = df.corr()
    plt.figure(figsize = (28, 26))

    # Heatmap of correlations
    sns.heatmap(df, cmap = plt.cm.RdYlBu_r, vmin = -0.25,fmt='.0g', annot = True, vmax = 0.6)
    plt.title('Correlation Heatmap')

In [ ]:
heatmap(prev)
heatmap(pos)
heatmap(ins)
heatmap(cc)

In [ ]:
 missing_data(prev)

# missing data gorsel

In [ ]:
fig = plt.figure(figsize=(18,6))
pa_dr = pd.DataFrame((pa.isnull().sum())*100/pa.shape[0]).reset_index()
pa_dr["type"] = "pre_app"
ax = sns.pointplot("index",0,data=pa_dr,hue="type")
plt.xticks(rotation =90,fontsize =7)
plt.title("Percentage of Missing values in application pre_app")
plt.ylabel("PERCENTAGE")
plt.xlabel("COLUMNS")
ax.set_facecolor("k")
fig.set_facecolor("lightgrey")

# previous_application

In [ ]:
pa = pd.read_csv('/kaggle/input/home-credit-default-risk/previous_application.csv')

preappcorr = pa.corr()
plt.figure(figsize = (28, 26))

# Heatmap of correlations
sns.heatmap(preappcorr, cmap = plt.cm.RdYlBu_r, vmin = -0.25,fmt='.0g', annot = True, vmax = 0.6)
plt.title('Correlation Heatmap')

missing_data(pa)

# pre_app char plot

import missingno as msno
msno.bar(pa);

# % 99 olan iki stunu (RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED) drop ediyoruz

pa.drop(pa[['RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED']],axis=1,inplace=True) 

# pre_app missin data nan to 0

df1 = pa[(pa.AMT_ANNUITY.isna() == True) & (pa.AMT_APPLICATION == 0 )].fillna(0)
df2 = pa[(pa.AMT_ANNUITY.isna() == True) & (pa.AMT_APPLICATION != 0 )]
df3 = pa[(pa.AMT_ANNUITY.isna() == False) & (pa.AMT_APPLICATION == 0 )]
df4 = pa[(pa.AMT_ANNUITY.isna() == False) & (pa.AMT_APPLICATION != 0 )]
pa = pd.concat([df1, df2, df3, df4])
display(missing_data(pa))
display(pa.shape)

msno.bar(pa);

pa['NAME_TYPE_SUITE'].value_counts()

pa['NAME_TYPE_SUITE'].isnull().sum()

pa['NAME_TYPE_SUITE'].replace(0 , 'Unaccompanied', inplace = True)
pa['NAME_TYPE_SUITE'].replace(np.nan , 'Unaccompanied', inplace = True)
pa['NAME_TYPE_SUITE'].value_counts()

msno.bar(pa);

pa['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
pa['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
pa['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
pa['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
pa['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

msno.bar(pa);

missing_data(pa)

# pre_app categoric values

pa.select_dtypes(include = [object]).columns

pa_cat = pa[['index','NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',
       'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY',
       'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE',
       'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION']]

pa_cat.select_dtypes(include = [object]).apply(pd.Series.nunique, axis = 0)

pa_cat = pa_cat.sort_index()
pa_cat

missing_data(pa_cat)

print('pa_cat shape:',pa_cat.shape)

# numeric values

pa_num = pa.select_dtypes(include = ['float64','int64'])
pa_num.columns

pa.groupby('SK_ID_CURR')['AMT_ANNUITY', 'AMT_APPLICATION',
       'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL'].mean()

# pre_app numeric values

pa_num = pa.select_dtypes(include = ['float64','int64'])
pa_drop = pa[['index', 'SK_ID_PREV', 'SK_ID_CURR', ]]
pa_num.drop(['SK_ID_CURR','SK_ID_PREV'],axis=1,inplace=True)
missing_data(pa_num)

pa_drop

!pip install ycimpute

np.array(pa_num)[0:5]

# pre_app numeric values to fill

from ycimpute.imputer import EM
pa_num_nnul = EM().complete(np.array(pa_num))


var_names = list(pa_num)
pa_num_nnul = pd.DataFrame(pa_num_nnul, columns = var_names)
missing_data(pa_num_nnul)

pre_app_num_corr = pa_num_nnul.corr()
plt.figure(figsize = (28, 26))
# Heatmap of correlations
sns.heatmap(pre_app_num_corr, cmap = plt.cm.RdYlBu_r, vmin = -0.25,fmt='.0g', annot = True, vmax = 0.6)
plt.title('Correlation Heatmap')

# OUTLIER

pa_num_nnul.shape

def iqr(df):
    IQR = df.describe().T
    IQR['lower'] = IQR['25%']-1.5*(IQR['75%']-IQR['25%'])
    IQR['upper'] = IQR['75%']+1.5*(IQR['75%']-IQR['25%'])
    upper_count = []
    i = 0
    for col in df:
        upper_count.append(len(df[df[col] > IQR.iloc[i][-1]]))
        i +=1
    print(upper_count)
    lower_count = []
    i = 0
    for col in df:
        lower_count.append(len(df[df[col] < IQR.iloc[i][-2]]))
        i +=1
    print(lower_count)
    IQR['lower_count'] = lower_count
    IQR['upper_count'] = upper_count
    IQR['out_percent'] = (IQR['upper_count'] + IQR['lower_count'])/IQR['count']
    return IQR

iqr(pa_num_nnul)

#  %10 dan daha buyukler baskilama ile outlier lardan temizliyoruz

pa_num_nnul.loc[pa_num_nnul['AMT_APPLICATION'] > 422820.00,'AMT_APPLICATION'] = 422820.00
pa_num_nnul.loc[pa_num_nnul['AMT_CREDIT'] > 504805.50,'AMT_CREDIT'] = 504805.50
pa_num_nnul.loc[pa_num_nnul['AMT_GOODS_PRICE'] > 422932.5,'AMT_GOODS_PRICE'] = 422932.5
pa_num_nnul.loc[pa_num_nnul['SELLERPLACE_AREA'] > 206.50,'SELLERPLACE_AREA'] = 206.50
pa_num_nnul.loc[pa_num_nnul['DAYS_LAST_DUE_1ST_VERSION'] > 870.00,'DAYS_LAST_DUE_1ST_VERSION'] = 870.00
pa_num_nnul.loc[pa_num_nnul['DAYS_LAST_DUE_1ST_VERSION'] < -1450.00,'DAYS_LAST_DUE_1ST_VERSION'] = -1450.00

iqr(pa_num_nnul)

pa.sort_values(by=['index'])

pa_cat

pa_num_nnul.sort_values(by=['index'])

pa_drop.sort_index()

result = pd.merge(pa_drop, pa_num_nnul)
result

missing_data(result)

result = pd.concat([panew, df_cat], axis=1)


result = result[['SK_ID_CURR', 'NAME_CONTRACT_TYPE','AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS',
       'DAYS_DECISION', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',
       'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY',
       'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE',
       'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY', 'CNT_PAYMENT',
       'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION', 'DAYS_FIRST_DRAWING',
       'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE',
       'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']]
result.shape

result

pre_app[pre_app.SK_ID_CURR == 175704]

result[result.SK_ID_CURR == 175704]

def aykiri(df):
    display(df.head())
    Q1=df.quantile(0.25)
    Q3=df.quantile(0.75)
    IQR=Q3-Q1
    alt_sinir=Q1-1.5*IQR
    ust_sinir=Q3+1.5*IQR
    sns.boxplot(x=df);
    aykiri_alt = (df < alt_sinir)
    aykiri_ust = (df > ust_sinir)
    #df[aykiri_alt] = alt_sinir
    #df[aykiri_ust] = ust_sinir
    print(alt_sinir,ust_sinir,(len(df[aykiri_ust])+len(df[aykiri_alt]))/len(df))

In [ ]:
preappcorr = pa.corr()
plt.figure(figsize = (28, 26))

# Heatmap of correlations
sns.heatmap(preappcorr, cmap = plt.cm.RdYlBu_r, vmin = -0.25,fmt='.0g', annot = True, vmax = 0.6)
plt.title('Correlation Heatmap')

In [ ]:
prev = pd.read_csv('previous_application.csv')
pos = pd.read_csv('POS_CASH_balance.csv')
ins = pd.read_csv('installments_payments.csv')
cc = pd.read_csv('credit_card_balance.csv')

# pre_app char plot

In [ ]:
import missingno as msno
msno.bar(pa);

# % 99 olan iki stunu (RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED) drop ediyoruz

In [ ]:
pa.drop(pa[['RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED']],axis=1,inplace=True) 

# pre_app missin data nan to 0

In [ ]:
df1 = pa[(pa.AMT_ANNUITY.isna() == True) & (pa.AMT_APPLICATION == 0 )].fillna(0)
df2 = pa[(pa.AMT_ANNUITY.isna() == True) & (pa.AMT_APPLICATION != 0 )]
df3 = pa[(pa.AMT_ANNUITY.isna() == False) & (pa.AMT_APPLICATION == 0 )]
df4 = pa[(pa.AMT_ANNUITY.isna() == False) & (pa.AMT_APPLICATION != 0 )]
pa = pd.concat([df1, df2, df3, df4])
display(missing_data(pa))
display(pa.shape)

In [ ]:
msno.bar(pa);

In [ ]:
pa['NAME_TYPE_SUITE'].value_counts()

In [ ]:
pa['NAME_TYPE_SUITE'].isnull().sum()

In [ ]:
pa['NAME_TYPE_SUITE'].replace(0 , 'Unaccompanied', inplace = True)
pa['NAME_TYPE_SUITE'].replace(np.nan , 'Unaccompanied', inplace = True)
pa['NAME_TYPE_SUITE'].value_counts()

In [ ]:
msno.bar(pa);

In [ ]:
pa['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
pa['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
pa['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
pa['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
pa['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

In [ ]:
msno.bar(pa);

In [ ]:
missing_data(pa)

# pre_app categoric values

In [ ]:
pa.select_dtypes(include = [object]).columns

In [ ]:
pa_cat = pa[['index','NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',
       'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY',
       'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE',
       'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION']]

In [ ]:
pa_cat.select_dtypes(include = [object]).apply(pd.Series.nunique, axis = 0)

In [ ]:
pa_cat = pa_cat.sort_index()
pa_cat

In [ ]:
missing_data(pa_cat)

In [ ]:
print('pa_cat shape:',pa_cat.shape)

# numeric values

In [ ]:
pa_num = pa.select_dtypes(include = ['float64','int64'])
pa_num.columns

In [ ]:
pa.groupby('SK_ID_CURR')['AMT_ANNUITY', 'AMT_APPLICATION',
       'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL'].mean()

# pre_app numeric values

In [ ]:
pa_num = pa.select_dtypes(include = ['float64','int64'])
pa_drop = pa[['index', 'SK_ID_PREV', 'SK_ID_CURR', ]]
pa_num.drop(['SK_ID_CURR','SK_ID_PREV'],axis=1,inplace=True)
missing_data(pa_num)

In [ ]:
pa_drop

In [ ]:
!pip install ycimpute

In [ ]:
np.array(pa_num)[0:5]

# pre_app numeric values to fill

In [ ]:
from ycimpute.imputer import EM
pa_num_nnul = EM().complete(np.array(pa_num))


In [ ]:
var_names = list(pa_num)
pa_num_nnul = pd.DataFrame(pa_num_nnul, columns = var_names)
missing_data(pa_num_nnul)

In [ ]:
pre_app_num_corr = pa_num_nnul.corr()
plt.figure(figsize = (28, 26))
# Heatmap of correlations
sns.heatmap(pre_app_num_corr, cmap = plt.cm.RdYlBu_r, vmin = -0.25,fmt='.0g', annot = True, vmax = 0.6)
plt.title('Correlation Heatmap')

# OUTLIER

In [ ]:
pa_num_nnul.shape

In [ ]:
def iqr(df):
    IQR = df.describe().T
    IQR['lower'] = IQR['25%']-1.5*(IQR['75%']-IQR['25%'])
    IQR['upper'] = IQR['75%']+1.5*(IQR['75%']-IQR['25%'])
    upper_count = []
    i = 0
    for col in df:
        upper_count.append(len(df[df[col] > IQR.iloc[i][-1]]))
        i +=1
    print(upper_count)
    lower_count = []
    i = 0
    for col in df:
        lower_count.append(len(df[df[col] < IQR.iloc[i][-2]]))
        i +=1
    print(lower_count)
    IQR['lower_count'] = lower_count
    IQR['upper_count'] = upper_count
    IQR['out_percent'] = (IQR['upper_count'] + IQR['lower_count'])/IQR['count']
    return IQR

In [ ]:
iqr(pa_num_nnul)

#  %10 dan daha buyukler baskilama ile outlier lardan temizliyoruz

In [ ]:
pa_num_nnul.loc[pa_num_nnul['AMT_APPLICATION'] > 422820.00,'AMT_APPLICATION'] = 422820.00
pa_num_nnul.loc[pa_num_nnul['AMT_CREDIT'] > 504805.50,'AMT_CREDIT'] = 504805.50
pa_num_nnul.loc[pa_num_nnul['AMT_GOODS_PRICE'] > 422932.5,'AMT_GOODS_PRICE'] = 422932.5
pa_num_nnul.loc[pa_num_nnul['SELLERPLACE_AREA'] > 206.50,'SELLERPLACE_AREA'] = 206.50
pa_num_nnul.loc[pa_num_nnul['DAYS_LAST_DUE_1ST_VERSION'] > 870.00,'DAYS_LAST_DUE_1ST_VERSION'] = 870.00
pa_num_nnul.loc[pa_num_nnul['DAYS_LAST_DUE_1ST_VERSION'] < -1450.00,'DAYS_LAST_DUE_1ST_VERSION'] = -1450.00

In [ ]:
iqr(pa_num_nnul)

In [ ]:
pa.sort_values(by=['index'])

In [ ]:
pa_cat

In [ ]:
pa_num_nnul.sort_values(by=['index'])

In [ ]:
pa_drop.sort_index()

In [ ]:
result = pd.merge(pa_drop, pa_num_nnul)
result

In [ ]:
missing_data(result)

In [ ]:
result = pd.concat([panew, df_cat], axis=1)


result = result[['SK_ID_CURR', 'NAME_CONTRACT_TYPE','AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS',
       'DAYS_DECISION', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',
       'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY',
       'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE',
       'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY', 'CNT_PAYMENT',
       'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION', 'DAYS_FIRST_DRAWING',
       'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE',
       'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']]
result.shape

In [ ]:
result

In [ ]:
pre_app[pre_app.SK_ID_CURR == 175704]

In [ ]:
result[result.SK_ID_CURR == 175704]

In [ ]:
def aykiri(df):
    display(df.head())
    Q1=df.quantile(0.25)
    Q3=df.quantile(0.75)
    IQR=Q3-Q1
    alt_sinir=Q1-1.5*IQR
    ust_sinir=Q3+1.5*IQR
    sns.boxplot(x=df);
    aykiri_alt = (df < alt_sinir)
    aykiri_ust = (df > ust_sinir)
    #df[aykiri_alt] = alt_sinir
    #df[aykiri_ust] = ust_sinir
    print(alt_sinir,ust_sinir,(len(df[aykiri_ust])+len(df[aykiri_alt]))/len(df))

# POS_CASH_balance

In [ ]:
posc = pd.read_csv('/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv')
display(posc)
missing_data(posc)

posc[(posc.CNT_INSTALMENT.isna() == True) & posc.CNT_INSTALMENT_FUTURE.isna() == False]

install = pd.read_csv('/kaggle/input/home-credit-default-risk/installments_payments.csv')
display(install)
missing_data(install)

crec = pd.read_csv('/kaggle/input/home-credit-default-risk/credit_card_balance.csv')
display(crec)
missing_data(crec)

crec[crec.AMT_DRAWINGS_ATM_CURRENT.isna() == True].sample(20)

crec[crec.AMT_DRAWINGS_ATM_CURRENT.isna() == False].sample(20)

In [ ]:
posc[(posc.CNT_INSTALMENT.isna() == True) & posc.CNT_INSTALMENT_FUTURE.isna() == False]

In [ ]:
install = pd.read_csv('/kaggle/input/home-credit-default-risk/installments_payments.csv')
display(install)
missing_data(install)

In [ ]:
crec = pd.read_csv('/kaggle/input/home-credit-default-risk/credit_card_balance.csv')
display(crec)
missing_data(crec)

In [ ]:
crec[crec.AMT_DRAWINGS_ATM_CURRENT.isna() == True].sample(20)

In [ ]:
crec[crec.AMT_DRAWINGS_ATM_CURRENT.isna() == False].sample(20)